In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import pylab as pl
from sklearn.metrics import confusion_matrix,accuracy_score
import pandas as pd

## as SIFT and SURF algorithms are function/feature which are not implemented
## This algorithm is patented and is excluded in this configuration
#!pip install opencv-contrib-python==4.4.0.44

In [ ]:
## exxtracting the made ziip file here 

from zipfile import ZipFile
file_name = '/content/Bikes.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
from zipfile import ZipFile
file_name = '/content/Horses.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
btrain_path="/content/Bikes"
bikes_name=os.listdir(btrain_path)

In [ ]:
#bikes_name

In [ ]:
type(bikes_name)

In [ ]:
htrain_path="/content/Horses"
horses_name=os.listdir(htrain_path)

In [ ]:
## declaring the class names .. order is important
class_names= ['horse','bike']

In [ ]:
type(class_names)

In [ ]:
## declaring the list og training and testing data 

image_paths=[]
himage_paths=[]
bimage_paths=[]

image_classes=[]

In [ ]:
def img_list(path):
    return (os.path.join(path,f) for f in os.listdir(path))

In [ ]:
for training_name in horses_name:
    class_path=os.path.join(htrain_path,training_name)
    #class_path=img_list(dir_)
    himage_paths.append(class_path)

In [ ]:
#himage_paths

In [ ]:
class_path

In [ ]:
for training_name in bikes_name:
    class_path=os.path.join(btrain_path,training_name)
    #class_path=img_list(dir_)
    bimage_paths.append(class_path)

In [ ]:
## totaL IMAGE PATHS list
image_paths = himage_paths + bimage_paths

In [ ]:
len(image_paths)

In [ ]:
type(image_paths)

In [ ]:
image_classes_0=[0]*(len(himage_paths))

In [ ]:
image_classes_1=[1]*(len(bimage_paths))

In [ ]:
image_classes=image_classes_0+image_classes_1

In [ ]:
image_classes

In [ ]:
len(image_classes)

In [ ]:
D=[]

In [ ]:
## creating list of image paths and its correponding labels 
for i in range(len(image_paths)):
    D.append((image_paths[i],image_classes[i]))

In [ ]:
## 1 -> bike 
## 0 -> horse

In [ ]:
len(D)

In [ ]:
dataset = D
random.shuffle(dataset)

In [ ]:
df = pd.DataFrame(dataset)
train = dataset[:140]
test = dataset[140:]

In [ ]:
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)

## defining train and test data 

In [ ]:
timage_paths = df_train[0]
y_train = df_train[1]
test_image_paths = df_test[0]
y_test = df_test[1]

In [ ]:
y_train

In [ ]:
type(timage_paths)

In [ ]:
df_train= pd.DataFrame(train)

In [ ]:
## just checking the creation
df_train[0]

In [ ]:
df_test = pd.DataFrame(test)

In [ ]:
df_train[1]

In [ ]:
train

In [ ]:
des_list=[]

In [ ]:

img = cv2.imread(image_paths[2], cv2.IMREAD_GRAYSCALE)

In [ ]:
# surf = cv2.xfeatures2d.SURF_create()
## checking the surf 

In [ ]:
#keypoints_surf, descriptors = surf.detectAndCompute(img, None)

In [ ]:
#img = cv2.drawKeypoints(img, keypoints, None)
#cv2.imshow("Image", img)

In [ ]:
orb=cv2.ORB_create()

## just checking for example

In [ ]:
im=cv2.imread(image_paths[5])

In [ ]:
plt.imshow(im)

In [ ]:
def draw_keypoints(vis, keypoints, color = (0, 255, 255)):
    for kp in keypoints:
            x, y = kp.pt
            plt.imshow(cv2.circle(vis, (int(x), int(y)), 2, color))

In [ ]:
kp = orb.detect(im,None)
kp, des = orb.compute(im, kp)
img=draw_keypoints(im,kp)

example ends :)

##Appending descriptors of the training images in list

In [ ]:
for image_pat in timage_paths:
    im=cv2.imread(image_pat)
    kp=orb.detect(im,None)
    keypoints,descriptor= orb.compute(im, kp)
    des_list.append((image_pat,descriptor))

In [ ]:
descriptors=des_list[0][1]
for image_path,descriptor in des_list[1:]:
    descriptors=np.vstack((descriptors,descriptor))

In [ ]:
descriptors.shape

In [ ]:
descriptors_float=descriptors.astype(float)

## K - means clustring on Descriptors

In [ ]:
from scipy.cluster.vq import kmeans,vq

In [ ]:
k=200
voc,variance=kmeans(descriptors_float,k,1)

##Creating histogram of training image

In [ ]:
im_features=np.zeros((len(timage_paths),k),"float32")
for i in range(len(timage_paths)):
    words,distance=vq(des_list[i][1],voc)
    for w in words:
        im_features[i][w]+=1

##Creating Classification Model with SVM

In [ ]:
from sklearn.svm import LinearSVC
clf=LinearSVC(max_iter=80000)
clf.fit(im_features,np.array(y_train))

In [ ]:
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=80000,
          multi_class='ovr', penalty='l2', tol=0.0001)

## testing the model and calculating the accuracy

In [ ]:
des_list_test=[]

In [ ]:
type(test)

In [ ]:
for image_pat in test_image_paths:
    image=cv2.imread(image_pat,1)
    kp=orb.detect(image,None)
    keypoints_test,descriptor_test= orb.compute(image, kp)
    des_list_test.append((image_pat,descriptor_test))

In [ ]:
len(des_list_test)

In [ ]:
len(test_image_paths)

In [ ]:
from scipy.cluster.vq import vq
test_features=np.zeros((len(test_image_paths),k),"float32")
for i in range(len(test_image_paths)):
    words,distance=vq(des_list_test[i][1],voc)
    for w in words:
        test_features[i][w]+=1

In [ ]:
test_features

In [ ]:
true_classes=[]
for i in y_test:
    if i==1:
        true_classes.append("Bike")
    else:
        true_classes.append("Horse")

In [ ]:
predict_classes=[]
for i in clf.predict(test_features):
    if i==1:
        predict_classes.append("Bike")
    else:
        predict_classes.append("Horse")

In [ ]:
print(true_classes)

In [ ]:
print(predict_classes)

In [ ]:
clf.predict(test_features)

In [ ]:
accuracy=accuracy_score(true_classes,predict_classes)
print(accuracy)